# Operaciones de strings vectorizadas

Uno de los puntos fuertes de Python es su relativa facilidad para manejar y manipular datos string.
Pandas se basa en esto y proporciona un conjunto completo de *operaciones de strings vectorizadas* que se convierten en una pieza esencial del tipo de manipulación necesaria cuando se trabaja con (léase: limpieza) datos del mundo real.
En esta sección, recorreremos algunas de las operaciones de string de Pandas, y luego echaremos un vistazo a su uso para limpiar parcialmente un conjunto de datos muy desordenado de recetas recogidas de Internet.

## Introducción a las operaciones de string de Pandas

Hemos visto en secciones anteriores cómo herramientas como NumPy y Pandas generalizan las operaciones aritméticas para que podamos realizar fácil y rápidamente la misma operación sobre muchos elementos de un array. Por ejemplo:

In [ ]:
import numpy as np
x = np.array([2, 3, 5, 7, 11, 13])
x * 2

array([ 4,  6, 10, 14, 22, 26])

Esta *vectorización* de las operaciones simplifica la sintaxis para operar con matrices de datos: ya no tenemos que preocuparnos por el tamaño o la forma de la matriz, sino sólo por la operación que queremos realizar.
Para las matrices de cadenas, NumPy no proporciona un acceso tan simple, y por lo tanto estás atascado usando una sintaxis de bucle más verbosa:

In [ ]:
data = ['peter', 'Paul', 'MARY', 'gUIDO']
[s.capitalize() for s in data]

['Peter', 'Paul', 'Mary', 'Guido']

Esto puede ser suficiente para trabajar con algunos datos, pero se romperá si hay valores perdidos.
Por ejemplo:

In [ ]:
data = ['peter', 'Paul', None, 'MARY', 'gUIDO']
[s.capitalize() for s in data]

Pandas incluye características para abordar tanto esta necesidad de operaciones de cadenas vectorizadas como para manejar correctamente los datos que faltan a través del atributo ``str`` de los objetos Pandas Series e Index que contienen cadenas.
Así, por ejemplo, supongamos que creamos una serie de Pandas con estos datos:

In [ ]:
import pandas as pd
names = pd.Series(data)
names

0    peter
1     Paul
2     None
3     MARY
4    gUIDO
dtype: object

Ahora podemos llamar a un único método que pondrá en mayúsculas todas las entradas, saltándose los valores que falten:

In [ ]:
names.str.capitalize()

0    Peter
1     Paul
2     None
3     Mary
4    Guido
dtype: object

Usando el tabulador en este atributo ``str`` se listarán todos los métodos de strings vectorizadas disponibles para Pandas.

## Tablas de métodos de strings de Pandas

Si tiene una buena comprensión de la manipulación de strings en Python, la mayor parte de la sintaxis de cadenas de Pandas es lo suficientemente intuitiva como para que probablemente sea suficiente con enumerar una tabla de métodos disponibles; empezaremos con eso aquí, antes de profundizar en algunas de las sutilezas.
Los ejemplos de esta sección utilizan la siguiente serie de nombres:

In [ ]:
monte = pd.Series(['Graham Chapman', 'John Cleese', 'Terry Gilliam',
                   'Eric Idle', 'Terry Jones', 'Michael Palin'])

### Métodos similares a los métodos de string de Python
Casi todos los métodos de cadena incorporados en Python se reflejan en un método de cadena vectorizado de Pandas. Aquí hay una lista de métodos ``str`` de Pandas que reflejan los métodos de cadena de Python:

|             |                  |                  |                  |
|-------------|------------------|------------------|------------------|
|``len()``    | ``lower()``      | ``translate()``  | ``islower()``    | 
|``ljust()``  | ``upper()``      | ``startswith()`` | ``isupper()``    | 
|``rjust()``  | ``find()``       | ``endswith()``   | ``isnumeric()``  | 
|``center()`` | ``rfind()``      | ``isalnum()``    | ``isdecimal()``  | 
|``zfill()``  | ``index()``      | ``isalpha()``    | ``split()``      | 
|``strip()``  | ``rindex()``     | ``isdigit()``    | ``rsplit()``     | 
|``rstrip()`` | ``capitalize()`` | ``isspace()``    | ``partition()``  | 
|``lstrip()`` |  ``swapcase()``  |  ``istitle()``   | ``rpartition()`` |

Fíjate en que tienen varios valores de retorno. Algunos, como ``lower()``, devuelven una serie de cadenas:

In [ ]:
monte.str.lower()

0    graham chapman
1       john cleese
2     terry gilliam
3         eric idle
4       terry jones
5     michael palin
dtype: object

Pero otros devuelven números:

In [ ]:
monte.str.len()

0    14
1    11
2    13
3     9
4    11
5    13
dtype: int64

O valores booleanos:

In [ ]:
monte.str.startswith('T')

0    False
1    False
2     True
3    False
4     True
5    False
dtype: bool

Otros devuelven lists u otros valores compuestos para cada elemento:

In [ ]:
monte.str.split()

0    [Graham, Chapman]
1       [John, Cleese]
2     [Terry, Gilliam]
3         [Eric, Idle]
4       [Terry, Jones]
5     [Michael, Palin]
dtype: object

Veremos más manipulaciones de este tipo de objeto de serie de listas cuando continuemos nuestra discusión.

### Métodos que utilizan expresiones regulares

Además, hay varios métodos que aceptan expresiones regulares para examinar el contenido de cada elemento de la cadena, y siguen algunas de las convenciones de la API del módulo incorporado ``re`` de Python:

| Método | Descripción |
|--------|-------------|
| ``match()`` | Llama a ``re.match()`` en cada elemento, devolviendo un booleano. |
| ``extract()`` | Llama a ``re.match()`` en cada elemento, devolviendo los grupos coincidentes como strings.|
| ``findall()`` | Llama a ``re.findall()``  en cada elemento |
| ``replace()`` | Reemplazar las ocurrencias del patrón con alguna otra cadena.|
| ``contains()`` | Llama a ``re.search()``  en cada elemento, devolviendo un booleano. |
| ``count()`` | Count occurrences of pattern|
| ``split()``   | Equivalente a ``str.split()``, pero acepta regexps |
| ``rsplit()`` | Equivalente a ``str.rsplit()``, pero acepta regexps |

Con ellas, se puede hacer una amplia gama de operaciones interesantes.
Por ejemplo, podemos extraer el nombre de cada uno pidiendo un grupo contiguo de caracteres al principio de cada elemento:

In [ ]:
monte.str.extract('([A-Za-z]+)', expand=False)

0     Graham
1       John
2      Terry
3       Eric
4      Terry
5    Michael
dtype: object

O podemos hacer algo más complicado, como encontrar todos los nombres que empiezan y terminan con una consonante, haciendo uso de los caracteres de expresión regular de inicio de cadena (``^``) y de fin de cadena (``$``):

In [ ]:
monte.str.findall(r'^[^AEIOU].*[^aeiou]$')

0    [Graham Chapman]
1                  []
2     [Terry Gilliam]
3                  []
4       [Terry Jones]
5     [Michael Palin]
dtype: object

La capacidad de aplicar de forma concisa expresiones regulares en las entradas de las "series" o "marcos de datos" abre muchas posibilidades de análisis y limpieza de datos.

### Métodos misceláneos
Finalmente, hay algunos métodos misceláneos que permiten otras operaciones convenientes:

| Método | Descripción | 
|--------|-------------|
| ``get()`` | Obtener cada elemento del índice|
| ``slice()`` | Cortar cada elemento|
| ``slice_replace()`` | Reemplazar la porción de cada elemento con el valor pasado|
| ``cat()``      |  Concatenar cadenas|
| ``repeat()`` | Repetir valores|
| ``normalize()`` | Devuelve la forma Unicode de la cadena|
| ``pad()`` | Añadir espacios en blanco a la izquierda, a la derecha o a ambos lados de las cadenas|
| ``wrap()`` | Dividir cadenas largas en líneas de longitud inferior a un ancho determinado.|
| ``join()`` |  Unir cadenas en cada elemento de la serie con el separador pasado|
| ``get_dummies()`` | Extraer las variables ficticias como un marco de datos |

#### Acceso vectorizado a elementos y corte

Las operaciones ``get()`` y ``slice()``, en particular, permiten el acceso a elementos vectorizados de cada array.
Por ejemplo, podemos obtener un corte de los tres primeros caracteres de cada matriz utilizando ``str.slice(0, 3)``.
Ten en cuenta que este comportamiento también está disponible a través de la sintaxis de indexación normal de Python, por ejemplo, ``df.str.slice(0, 3)`` es equivalente a ``df.str[0:3]``:

In [ ]:
monte.str[0:3]

0    Gra
1    Joh
2    Ter
3    Eri
4    Ter
5    Mic
dtype: object

La indexación a través de ``df.str.get(i)`` y ``df.str[i]`` es también similar.

Estos métodos ``get()`` y ``slice()`` también permiten acceder a los elementos de los arrays devueltos por ``split()``.
Por ejemplo, para extraer el último nombre de cada entrada, podemos combinar ``split()`` y ``get()``:

In [ ]:
monte.str.split().str.get(-1)

0    Chapman
1     Cleese
2    Gilliam
3       Idle
4      Jones
5      Palin
dtype: object

#### Variables indicadoras

Otro método que requiere un poco de explicación extra es el método ``get_dummies()``.
Es útil cuando tus datos tienen una columna que contiene algún tipo de indicador codificado.
Por ejemplo, podemos tener un conjunto de datos que contenga información en forma de códigos, como A="nacido en América", B="nacido en el Reino Unido", C="le gusta el queso", D="le gusta el spam":

In [ ]:
full_monte = pd.DataFrame({'name': monte,
                           'info': ['B|C|D', 'B|D', 'A|C',
                                    'B|D', 'B|C', 'B|C|D']})
full_monte

,info,name
0,B|C|D,Graham Chapman
1,B|D,John Cleese
2,A|C,Terry Gilliam
3,B|D,Eric Idle
4,B|C,Terry Jones
5,B|C|D,Michael Palin


La rutina ``get_dummies()`` permite dividir rápidamente estas variables indicadoras en un ``DataFrame``:

In [ ]:
full_monte['info'].str.get_dummies('|')

,A,B,C,D
0,0,1,1,1
1,0,1,0,1
2,1,0,1,0
3,0,1,0,1
4,0,1,1,0
5,0,1,1,1


Con estas operaciones como bloques de construcción, puede construir un rango interminable de procedimientos de procesamiento de cadenas cuando limpie sus datos.

No vamos a profundizar en estos métodos aquí, pero le animo a leer ["Working with Text Data"](http://pandas.pydata.org/pandas-docs/stable/text.html) en la documentación online de Pandas, o a consultar los recursos listados en [Más recursos](03.13-Más-recursos.ipynb).

## Ejemplo: Database Recipe

Estas operaciones de strings vectorizadas son muy útiles en el proceso de limpieza de datos desordenados del mundo real.

Aquí voy a ver un ejemplo de ello, utilizando una Database abierta Recipe compilada a partir de varias fuentes en la web.
Nuestro objetivo será analizar los datos de las recetas en listas de ingredientes, para poder encontrar rápidamente una receta basada en algunos ingredientes que tengamos a mano.

Los scripts utilizados para compilar esto se pueden encontrar en https://github.com/fictivekin/openrecipes, y el enlace a la versión actual de la base de datos se encuentra allí también.

A partir de la primavera de 2016, esta base de datos ocupa unos 30 MB, y se puede descargar y descomprimir con estos comandos:

In [ ]:
# !curl -O http://openrecipes.s3.amazonaws.com/recipeitems-latest.json.gz
# !gunzip recipeitems-latest.json.gz

La base de datos está en formato JSON, así que probaremos ``pd.read_json`` para leerla:

In [ ]:
try:
    recipes = pd.read_json('recipeitems-latest.json')
except ValueError as e:
    print("ValueError:", e)

ValueError: Trailing data


¡Uy! Obtenemos un ``ValueError`` mencionando que hay "trailing data".
Buscando el texto de este error en Internet, parece que se debe a que se utiliza un archivo en el que *cada línea* es en sí misma un JSON válido, pero el archivo completo no lo es.
Vamos a comprobar si esta interpretación es cierta:

In [ ]:
with open('recipeitems-latest.json') as f:
    line = f.readline()
pd.read_json(line).shape

(2, 12)

Sí, aparentemente cada línea es un JSON válido, así que tendremos que encadenarlas.
Una forma de hacerlo es construir una representación de cadena que contenga todas estas entradas JSON, y luego cargar todo con ``pd.read_json``:

In [ ]:
# read the entire file into a Python array
with open('recipeitems-latest.json', 'r') as f:
    # Extract each line
    data = (line.strip() for line in f)
    # Reformat so each line is the element of a list
    data_json = "[{0}]".format(','.join(data))
# read the result as a JSON
recipes = pd.read_json(data_json)

In [ ]:
recipes.shape

(173278, 17)

Vemos que hay casi 200.000 recetas y 17 columnas.
Echemos un vistazo a una fila para ver lo que tenemos:

In [ ]:
recipes.iloc[0]

_id                                {'$oid': '5160756b96cc62079cc2db15'}
cookTime                                                          PT30M
creator                                                             NaN
dateModified                                                        NaN
datePublished                                                2013-03-11
description           Late Saturday afternoon, after Marlboro Man ha...
image                 http://static.thepioneerwoman.com/cooking/file...
ingredients           Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
name                                    Drop Biscuits and Sausage Gravy
prepTime                                                          PT10M
recipeCategory                                                      NaN
recipeInstructions                                                  NaN
recipeYield                                                          12
source                                                  thepione

Hay mucha información, pero gran parte de ella está muy desordenada, como es típico de los datos extraídos de la web.
En particular, la lista de ingredientes está en formato de cadena; vamos a tener que extraer cuidadosamente la información que nos interesa.
Empecemos por examinar más detenidamente los ingredientes:

In [ ]:
recipes.ingredients.str.len().describe()

count    173278.000000
mean        244.617926
std         146.705285
min           0.000000
25%         147.000000
50%         221.000000
75%         314.000000
max        9067.000000
Name: ingredients, dtype: float64

The ingredient lists average 250 characters long, with a minimum of 0 and a maximum of nearly 10,000 characters!

Just out of curiousity, let's see which recipe has the longest ingredient list:

In [ ]:
recipes.name[np.argmax(recipes.ingredients.str.len())]

'Carrot Pineapple Spice &amp; Brownie Layer Cake with Whipped Cream &amp; Cream Cheese Frosting and Marzipan Carrots'

Desde luego, parece una receta complicada.

Podemos hacer otras exploraciones agregadas; por ejemplo, veamos cuántas de las recetas son de comida para el desayuno:

In [ ]:
recipes.description.str.contains('[Bb]reakfast').sum()

3524

O cuántas de las recetas incluyen la canela como ingrediente:

In [ ]:
recipes.ingredients.str.contains('[Cc]innamon').sum()

10526

Incluso podríamos mirar si alguna receta escribe mal el ingrediente como "cinamón":

In [ ]:
recipes.ingredients.str.contains('[Cc]inamon').sum()

11

Este es el tipo de exploración de datos esencial que es posible con las herramientas de cadena de Pandas.
Es en este tipo de datos en los que Python realmente destaca.

### Un simple recomendador de recetas

Vayamos un poco más allá, y empecemos a trabajar en un sencillo sistema de recomendación de recetas: dada una lista de ingredientes, encontrar una receta que utilice todos esos ingredientes.
Aunque conceptualmente es sencillo, la tarea se complica por la heterogeneidad de los datos: no hay una operación fácil, por ejemplo, para extraer una lista limpia de ingredientes de cada fila.
Así que haremos un poco de trampa: empezaremos con una lista de ingredientes comunes, y simplemente buscaremos para ver si están en la lista de ingredientes de cada receta.
Para simplificar, vamos a limitarnos a las hierbas y especias por el momento:

In [ ]:
spice_list = ['salt', 'pepper', 'oregano', 'sage', 'parsley',
              'rosemary', 'tarragon', 'thyme', 'paprika', 'cumin']

A continuación, podemos construir un ``DataFrame`` booleano formado por los valores True y False, que indica si este ingrediente aparece en la lista:

In [ ]:
import re
spice_df = pd.DataFrame(dict((spice, recipes.ingredients.str.contains(spice, re.IGNORECASE))
                             for spice in spice_list))
spice_df.head()

,cumin,oregano,paprika,parsley,pepper,rosemary,sage,salt,tarragon,thyme
0,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,True,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False


Ahora, como ejemplo, digamos que nos gustaría encontrar una receta que utilice perejil, pimentón y estragón.
Podemos calcular esto muy rápidamente usando el método ``query()`` de ``DataFrame``, discutido en [Alto rendimiento de Pandas: ``eval()`` y ``query()``](03.12-Rendimiento-Eval-y-Query.ipynb):

In [ ]:
selection = spice_df.query('parsley & paprika & tarragon')
len(selection)

10

Sólo encontramos 10 recetas con esta combinación; utilicemos el índice devuelto por esta selección para descubrir los nombres de las recetas que tienen esta combinación:

In [ ]:
recipes.name[selection.index]

2069      All cremat with a Little Gem, dandelion and wa...
74964                         Lobster with Thermidor butter
93768      Burton's Southern Fried Chicken with White Gravy
113926                     Mijo's Slow Cooker Shredded Beef
137686                     Asparagus Soup with Poached Eggs
140530                                 Fried Oyster Po’boys
158475                Lamb shank tagine with herb tabbouleh
158486                 Southern fried chicken in buttermilk
163175            Fried Chicken Sliders with Pickles + Slaw
165243                        Bar Tartine Cauliflower Salad
Name: name, dtype: object

Ahora que hemos reducido nuestra selección de recetas por un factor de casi 20.000, estamos en condiciones de tomar una decisión más informada sobre lo que nos gustaría cocinar para la cena.

### Ir más allá con las recetas

Esperemos que este ejemplo le haya dado un poco de sabor (¡ba-dum!) para los tipos de operaciones de limpieza de datos que son eficientemente habilitados por los métodos de cadena de Pandas.
Por supuesto, construir un sistema de recomendación de recetas muy robusto requeriría un *mucho* trabajo más.
Extraer las listas completas de ingredientes de cada receta sería una parte importante de la tarea; desgraciadamente, la gran variedad de formatos utilizados hace que este proceso sea relativamente largo.
Esto apunta a la obviedad de que en la ciencia de datos, la limpieza y la manipulación de los datos del mundo real a menudo constituyen la mayor parte del trabajo, y Pandas proporciona las herramientas que pueden ayudarle a hacer esto de manera eficiente.